In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn import preprocessing    
le = preprocessing.LabelEncoder()
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

train = pd.read_csv('../input/spaceship-titanic/train.csv', index_col='PassengerId')
test = pd.read_csv('../input/spaceship-titanic/test.csv', index_col='PassengerId')

train.head()

In [ ]:
y=train.Transported
X=train.drop(['Transported'], axis=1)

In [ ]:
sns.boxplot(x='FoodCourt',data=X)

I begin EDA by searching for outliers among quantitative variables. Age is left out because the range of values isn't as large as the others. Boxplot is useful for visualizing outliers

In [ ]:
sns.boxplot(x='Transported', y='ShoppingMall',data=train)

In [ ]:
sns.boxplot(x='Spa',data=X)

In [ ]:
sns.boxplot(x='VRDeck',data=train)

In [ ]:
sns.boxplot(x='RoomService',data=train)

In [ ]:
X.loc[X['FoodCourt'] > 20000, 'FoodCourt'] = X.loc[X['FoodCourt'] < 20000, 'FoodCourt'].mean()
X.loc[X['ShoppingMall'] > 10000, 'ShoppingMall'] = X.loc[X['ShoppingMall'] < 10000, 'ShoppingMall'].mean()
X.loc[X['Spa'] > 20000, 'Spa'] = X.loc[X['Spa'] < 20000, 'Spa'].mean()

X.loc[X['VRDeck'] > 20000, 'VRDeck'] = X.loc[X['VRDeck'] < 20000, 'VRDeck'].mean()

X.loc[X['RoomService'] > 10000, 'RoomService'] = X.loc[X['RoomService'] < 10000, 'RoomService'].mean()


After visualizing outliers, I edited to control the skew

In [ ]:
sns.histplot(x='Age', hue='Transported', data=train, bins=20, multiple="stack", kde=True)

The histogram illustrates that the rate of transport. The following is to control the skew

In [ ]:
X['Under15'] = X['Age'].apply(lambda x: 1 if x < 15 else 0)
test['Under15'] = test['Age'].apply(lambda x: 1 if x < 15 else 0)
X = X.drop(['Age'], axis=1)
test = test.drop(['Age'], axis=1)

In [ ]:
sns.histplot(x="HomePlanet",hue="Destination", data=train
             , multiple="stack", kde=True)

This histogram illustrates a higher prevalence of passengers from Earth, with Trappist-1e as the most popular destination

In [ ]:
sns.countplot(x="CryoSleep",hue="Transported", data=train)

Diagram illustrates recipients of CryoSleep had higher rate of transport

In [ ]:
sns.swarmplot(x=train["CryoSleep"],y=train['Age'],hue=train["Transported"],s=0.5)

The diagram shows CryoSleep recipients of all ages had a higher rate of transport while nonrecipients had highest prevalence of transport at the 0-15 age range.

In [ ]:
def CabinExtract(df):
    df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)    
    df['Num'] = df['Num'].astype(float)
    
    df['Deck']= df['Deck'].fillna('G')
    df['Num']= df['Num'].fillna(df['Num'].mean())
    df['Side']= df['Side'].fillna('S')
    
             
    df['Deck'] = le.fit_transform(df['Deck'])
    df['Side'] = le.fit_transform(df['Side'])
    
    return df

X = CabinExtract(X)
test = CabinExtract(test)

Function to split Cabin data into its 3 relevant components.

In [ ]:
from sklearn import preprocessing    
le = preprocessing.LabelEncoder()
def converting_column(df,col,convertType):
    if convertType=='OHE':
        one_hot = pd.get_dummies(df[col],prefix=col)
        df = df.drop(col,axis = 1)
        df = df.join(one_hot)
        return df
    else:
        df[col] = le.fit_transform(df[col])
        return df
X = converting_column(X,'HomePlanet','OHE')
test = converting_column(test,'HomePlanet','OHE')
X = converting_column(X,'Destination','FTR')
test = converting_column(test,'Destination','FTR')
X = converting_column(X,'CryoSleep','FTR')
test = converting_column(test,'CryoSleep','FTR')
X = converting_column(X,'VIP','FTR')
test = converting_column(test,'VIP','FTR')

Function to encode qualitative variables.

In [ ]:
Amenities = ['RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
X['Amenities'] = X[Amenities].sum(axis=1)
test['Amenities'] = test[Amenities].sum(axis=1)
X.drop(X[Amenities], inplace=True, axis=1)
test.drop(test[Amenities], inplace=True, axis=1)

X.head(5)

In [ ]:
X = X.drop(['VIP','Name','HomePlanet_Mars'], axis=1)
test = test.drop(['VIP','Name','HomePlanet_Mars'], axis=1)

Feature engineering to add up total money passenger spent on amenities and present as one feature.

In [ ]:


X_train, X_val, y_train, y_val = train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)


categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_t = X_train[my_cols].copy()
X_v = X_val[my_cols].copy()
X_test = test[my_cols].copy()

Split training data to train and validate. Define qualitative and quantitative columns for pipeline

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from lightgbm import LGBMClassifier


model = LGBMClassifier(n_estimators=975, min_child_samples=5, max_depth=8, learning_rate=0.01,feature_fraction=0.6, 
                       bagging_freq=5, bagging_fraction= 0.9,objective ='binary')







Import libraries and define model. Chose a classifier due to categorical label. Model candidates were: Light Gradient Boosting Model, Random Forest Classifier, and XG Classifier

In [ ]:

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


from sklearn.metrics import log_loss
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

Construct pipeline, imported log_loss as loss function due to nature of output

In [ ]:
my_pipeline.fit(X_train,y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_log_loss')

print("Log_Loss:\n", scores)

In [ ]:
print("Average Log_Loss score (across experiments):")
print(scores.mean())

I conducted cross validation, and calculated average score. 

I tried all three models mentioned earlier. The LGBM received highest log_loss score which is why I selected, however contest results differences between LGBM and XG Classifier are negligible. Random Forest Classifier was lower

In [ ]:
test_preds=my_pipeline.predict(test)

                            
data_new1 = test_preds.astype(bool) 
print(data_new1)



In [ ]:

output = pd.DataFrame({'PassengerId': test.index,
                       'Transported': data_new1})
output.to_csv('submissionnew1.csv', index=False)